In [1]:
# Database
from database.getter import get_jobs_by_instance, get_jssp_by_job_ids

# Utils
from src.utils.logging import print_experiment_log
from src.utils.rolling_process.rolling_filter import get_current_jobs, filter_current_jssp

# Solver
from src.solvers.converter import get_job_ops_dict, get_times_dict
from src.solvers.cp.lateness import solve_jssp_lateness_with_start_deviation_minimization

# Simulation
from src.simulation.ProductionSimulation import ProductionSimulation

In [2]:
# experiments params
experiment_main_pct: float = (1/3)    # 33.3 % Lateness; 66.6 % Deviation Penalty
experiment_w_t: int = 10
experiment_w_e: int = 2
experiment_w_first: int = 1
experiment_sim_sigma: float = 0.2

# fix params
shift_length = 1440

In [3]:
# Hard termination condition for schedulers
max_time = 60*3     # 3 min

# Simulation
simulation = ProductionSimulation(shift_length=shift_length, sigma=experiment_sim_sigma)

In [4]:
df_jobs_times = get_jobs_by_instance("Fisher and Thompson 10x10")
df_jobs_times

✅ 993 jobs found for instance 'Fisher and Thompson 10x10'.


,Job,Routing_ID,Arrival,Ready Time,Deadline
0,J25-0000,8,79,1440,3660
1,J25-0001,6,129,1440,2338
2,J25-0002,0,198,1440,3744
3,J25-0003,2,243,1440,3379
4,J25-0004,3,287,1440,4856
...,...,...,...,...,...
988,J25-0988,0,71217,72000,73988
989,J25-0989,6,71417,72000,73911
990,J25-0990,7,71807,72000,74446
991,J25-0991,2,71816,72000,74554


In [5]:
job_ids = df_jobs_times["Job"].tolist()
df_jssp = get_jssp_by_job_ids(job_ids=job_ids)
df_jssp

✅ Retrieved JSSP data for 993 jobs with 9930 operations.


,Job,Routing_ID,Operation,Machine,Processing Time
0,J25-0000,8,0,M00,76
1,J25-0000,8,1,M01,69
2,J25-0000,8,2,M03,76
3,J25-0000,8,3,M05,51
4,J25-0000,8,4,M02,85
...,...,...,...,...,...
9925,J25-0992,6,5,M05,21
9926,J25-0992,6,6,M09,32
9927,J25-0992,6,7,M08,89
9928,J25-0992,6,8,M07,30


## Day 1

In [6]:
day_numb = 1
day_start = shift_length*day_numb
day_end = day_start + shift_length
print(f"Tag {day_numb:02d}: [{day_start}, {day_end})")

Tag 01: [1440, 2880)


In [7]:
# new and unfinished jobs
df_jobs_times_current = get_current_jobs(
    df_jobs_times, df_previous_not_started = None, ready_time=day_start
)
df_jobs_times_current

,Job,Routing_ID,Arrival,Ready Time,Deadline
0,J25-0000,8,79,1440,3660
1,J25-0001,6,129,1440,2338
2,J25-0002,0,198,1440,3744
3,J25-0003,2,243,1440,3379
4,J25-0004,3,287,1440,4856
5,J25-0005,1,498,1440,3106
6,J25-0006,9,790,1440,3539
7,J25-0007,5,925,1440,2929
8,J25-0008,4,988,1440,3541
9,J25-0009,7,1004,1440,3578


In [8]:
df_jssp_current = filter_current_jssp(df_jssp, df_jobs_times_current)
df_jssp_current

,Job,Routing_ID,Operation,Machine,Processing Time
0,J25-0000,8,0,M00,76
1,J25-0000,8,1,M01,69
2,J25-0000,8,2,M03,76
3,J25-0000,8,3,M05,51
4,J25-0000,8,4,M02,85
...,...,...,...,...,...
155,J25-0015,5,5,M09,72
156,J25-0015,5,6,M00,47
157,J25-0015,5,7,M06,65
158,J25-0015,5,8,M04,6


### 1. Scheduling

In [1]:
job_ops_dict = get_job_ops_dict(df_jssp_current)
job_ops_dict

NameError: name 'get_job_ops_dict' is not defined

In [ ]:
times_dict = get_times_dict(df_jobs_times_current)

In [10]:
from src.utils.logging import print_experiment_log
from src.solvers.cp.lateness import solve_jssp_lateness_with_start_deviation_minimization

schedule, experiment_log  = solve_jssp_lateness_with_start_deviation_minimization(
    job_ops=job_ops_dict,
    times_dict=times_dict,
    previous_schedule=None,
    active_ops=None,
    w_t= experiment_w_t,
    w_e=experiment_w_e,
    w_first=experiment_w_first,
    main_pct=experiment_main_pct,
    duration_buffer_factor = 2,
    schedule_start=day_start, msg=False,
    solver_time_limit=max_time, solver_relative_gap_limit=0.05
    )

print_experiment_log(experiment_log)


===== EXPERIMENT LOG SUMMARY =====
[EXPERIMENT_INFO]
  total_number_of_operations                        : 160
  number_of_operations_with_previous_schedule       : 0
  number_of_active_operation_to_consider            : 0
  schedule_start                                    : 1440
[EXPERIMENT_CONFIG]
  main_pct                                          : 1.0
  w_t                                               : 10
  w_e                                               : 2
  w_first                                           : 1
  deviation_type                                    : start
  solver_time_limit                                 : 180
  solver_relative_gap_limit                         : 0.05
[MODEL_INFO]
  number_of_variables                               : 421
  number_of_constraints                             : 575
  number_of_deviation_terms                         : 0
[SOLVER_INFO]
  status                                            : OPTIMAL
  objective_value               